<a href="https://colab.research.google.com/github/chiyanglin-AStar/science_coding/blob/main/QM_num_demo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Numerical Methods of Quantum Mechanics

Sure, let’s go over how each of these methods could be applied to the hydrogen atom (or, more precisely, a simplified hydrogen-like atom model). The hydrogen atom is a single electron in the Coulomb field of a proton, which makes it a great example to demonstrate quantum mechanical simulation methods.

To make this accessible, I'll simplify the problem by focusing on the electron in a fixed Coulomb potential (ignoring the nucleus's movement). We can explore the **ground-state** properties and **time evolution** of the electron in the hydrogen atom potential, $V(r) = -\frac{e^2}{4 \pi \epsilon_0 r}$, where $e$ is the electron charge, $\epsilon_0$ is the permittivity, and $r$ is the distance from the nucleus.

For simplicity, let’s break it down with each method.


### 1. **Matrix Methods**

Using matrix methods for the hydrogen atom involves representing the **Hamiltonian** as a matrix and solving for the **eigenvalues** and **eigenvectors**, which correspond to the energy levels and stationary states of the electron.

#### Key Steps:
1. **Discretize** the radial part of the wavefunction (1D in $r$) or full 3D space.
2. **Construct the Hamiltonian** matrix, which includes the kinetic and potential energy terms.
3. Use **matrix diagonalization** to find the eigenvalues (energies) and eigenvectors (wavefunctions).

Here’s a basic pseudocode outline in Python:

In [ ]:
import numpy as np
from scipy.linalg import eigh  # For matrix diagonalization

# Constants (simplified)
num_points = 100  # Number of points in the radial direction
r_max = 20.0      # Max radius to consider
dr = r_max / num_points
r = np.linspace(dr, r_max, num_points)

# Constructing the Hamiltonian matrix
kinetic = np.diag(-2 * np.ones(num_points)) + np.diag(np.ones(num_points - 1), 1) + np.diag(np.ones(num_points - 1), -1)
kinetic = -kinetic / (2 * dr**2)  # Discrete Laplacian for kinetic energy

potential = -1 / r  # Coulomb potential, V(r) = -1/r
potential = np.diag(potential)

# Hamiltonian matrix
H = kinetic + potential

# Solve for eigenvalues and eigenvectors
eigenvalues, eigenvectors = eigh(H)
print("Ground-state energy:", eigenvalues[0])
print("Ground-state wavefunction:", eigenvectors[:, 0])

This code finds the ground state energy and wavefunction by diagonalizing the Hamiltonian matrix. The ground state should approximate \(-13.6\ \text{eV}\), the known ground-state energy of the hydrogen atom.

### 2. **Wavefunction Methods**

Wavefunction methods involve solving the **time-dependent Schrödinger equation** for the hydrogen atom. This is challenging because the Coulomb potential creates non-trivial radial dependencies, but we can still get approximate results in 1D.

A simple approach is to use a **finite-difference time-dependent Schrödinger equation (TDSE)** solver.

#### Key Steps:
1. Initialize a wavefunction (e.g., in the ground state or another initial guess).
2. Use a **finite difference** or **Crank-Nicolson** method to iteratively solve the TDSE over small time steps.

Here’s a rough outline of how you might simulate time evolution:


## 3. **Density Matrix Methods**

The **density matrix** is useful for studying open quantum systems or mixed states, but we can also use it for the hydrogen atom to calculate **observable properties** or track decoherence effects.

#### Key Steps:
1. Create the initial density matrix, $\rho = |\psi\rangle \langle \psi|$, where $\psi$ is the ground-state wavefunction.
2. Evolve $\rho$ over time using $\rho(t) = U \rho U^\dagger$, where $U$ is the time-evolution operator.
3. Calculate properties like expected energy from $\text{Tr}(\rho H)$.

Example pseudocode:

In [ ]:
# Construct initial density matrix for the ground state
rho_0 = np.outer(eigenvectors[:, 0], np.conj(eigenvectors[:, 0]))

# Time evolution of density matrix using U = exp(-i * H * dt)
from scipy.linalg import expm
U = expm(-1j * H * dt)  # Time evolution operator

# Evolve the density matrix over time
rho_t = U @ rho_0 @ U.conj().T  # After one time step, repeat for longer evolution

# Calculate the expected energy
expected_energy = np.trace(rho_t @ H)
print("Expected energy:", expected_energy)

This example evolves the density matrix over one time step and calculates the expected energy. In practice, you would evolve over more time steps.

### 4. **Monte Carlo Methods**

Monte Carlo methods can approximate the **ground-state energy** or simulate properties using **random sampling**. For example, Quantum Monte Carlo (QMC) techniques are often used for more complex atomic systems, but here’s a basic example to estimate the ground state of the hydrogen atom using **Variational Monte Carlo (VMC)**.

#### Key Steps:
1. Choose a **trial wavefunction**, e.g., a simple 1s orbital \(\psi(r) = e^{-\alpha r}\), with variational parameter \(\alpha\).
2. Use **Metropolis sampling** to compute the **expectation value of the energy** with respect to the trial wavefunction.
3. Optimize \(\alpha\) to minimize the energy.

In [ ]:
import random

# Trial wavefunction and local energy
def trial_wavefunction(r, alpha):
    return np.exp(-alpha * r)

def local_energy(r, alpha):
    return -alpha**2 / 2 + alpha / r  # Simplified energy of 1s orbital

# Parameters
alpha = 1.0  # Initial guess for alpha
num_samples = 10000  # Number of samples

# Monte Carlo sampling
energy_samples = []
for _ in range(num_samples):
    # Sample r using trial wavefunction as the probability distribution
    r = -np.log(1 - random.random()) / alpha  # Sample r based on exponential distribution

    # Calculate local energy and accumulate
    energy_samples.append(local_energy(r, alpha))

# Calculate average energy
average_energy = np.mean(energy_samples)
print("Estimated ground-state energy:", average_energy)

This code approximates the ground-state energy by averaging the local energy of the hydrogen atom across many random samples. With a correctly chosen trial function and optimized $\alpha$, it should approximate the correct energy.

### Summary

Here's how each method approaches the hydrogen atom problem:

- **Matrix Methods**: Discretize the Hamiltonian, build a matrix representation, and use matrix diagonalization to find the energy levels and wavefunctions.
- **Wavefunction Methods**: Use time-dependent methods to evolve a wavefunction directly.
- **Density Matrix Methods**: Use the density matrix formalism to evolve the state and compute properties, suitable for mixed states or open quantum systems.
- **Monte Carlo Methods**: Use sampling techniques (such as Variational Monte Carlo) to estimate ground-state properties by minimizing the energy with respect to a trial wavefunction.

These approaches require more sophistication to handle the full 3D hydrogen atom, but they illustrate the principles involved. Let me know if you'd like more details on any specific method!